In [ ]:
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

In [ ]:
# this notebook intended for internal testing purpose.

Apache Beam can be installed directly from the main branch of https://github.com/apache/beam or can be installed via `pip install apache_beam>=2.45.0`

In [ ]:
!git clone https://github.com/apache/beam.git
!pip install -r beam/sdks/python/build-requirements.txt
!pip install -e beam/sdks/python[gcp]
!pip install matplotlib
!pip install pandas

Import necessary dependencies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from apache_beam.testing.load_tests import load_test_metrics_utils
from apache_beam.testing.load_tests.load_test_metrics_utils import BigQueryMetricsFetcher

In [ ]:
bq_project = 'apache-beam-testing'
bq_dataset = '<bq-dataset-name>' # sample value: beam_run_inference
bq_table = '<bq-table>' # sample value: torch_inference_imagenet_results_resnet152
metric_name = '<perf-alerted-metric-name>' # sample value: mean_load_model_latency_milli_secs

query = f"""
      SELECT *
      FROM {bq_project}.{bq_dataset}.{bq_table}
      WHERE CONTAINS_SUBSTR(({load_test_metrics_utils.METRICS_TYPE_LABEL}), '{metric_name}')
      ORDER BY {load_test_metrics_utils.SUBMIT_TIMESTAMP_LABEL} DESC
      LIMIT 30
    """


If the performance/load test store the results in BigQuery using this [schema](https://github.com/apache/beam/blob/83679216cce2d52dbeb7e837f06ca1d57b31d509/sdks/python/apache_beam/testing/load_tests/load_test_metrics_utils.py#L66),
then fetch the metric_values for a `metric_name` for the last `30` runs and display a plot using matplotlib.


In [ ]:
big_query_metrics_fetcher = BigQueryMetricsFetcher()
metric_data: pd.DataFrame = big_query_metrics_fetcher.fetch(query=query)

In [ ]:
# sort the data to view it in chronological order.
metric_data.sort_values(
      by=[load_test_metrics_utils.SUBMIT_TIMESTAMP_LABEL], inplace=True)

In [ ]:
metric_data.plot(x=load_test_metrics_utils.SUBMIT_TIMESTAMP_LABEL,
                 y=load_test_metrics_utils.VALUE_LABEL)
plt.show()